In [ ]:
%install-location $cwd/swift-install
%install '.package(path: "$cwd/FastaiNotebook_08a_heterogeneous_dictionary")' FastaiNotebook_08a_heterogeneous_dictionary

Installing packages:
	.package(path: "/home/ubuntu/fastai_docs/dev_swift/FastaiNotebook_08a_heterogeneous_dictionary")
		FastaiNotebook_08a_heterogeneous_dictionary
With SwiftPM flags: []
Working in: /tmp/tmpj_ch4_ln/swift-install
Compile Swift Module 'FastaiNotebook_08a_heterogeneous_dictionary' (13 sources)
Compile Swift Module 'jupyterInstalledPackages' (1 sources)
Linking ./.build/x86_64-unknown-linux/debug/libjupyterInstalledPackages.so
Initializing Swift...
Installation complete!


In [ ]:
// export
import Path
import TensorFlow

In [ ]:
import FastaiNotebook_08a_heterogeneous_dictionary

In [ ]:
%include "EnableIPythonDisplay.swift"
IPythonDisplay.shell.enable_matplotlib("inline")

('inline', 'module://ipykernel.pylab.backend_inline')


## Load data

Will train on imagenette when it's possible without OOM.

In [ ]:
//let path = downloadImagette()

In [ ]:
//let il = ItemList(fromFolder: path, extensions: ["jpeg", "jpg"])
//let sd = SplitData(il, fromFunc: {grandParentSplitter(fName: $0, valid: "val")})
//var (procItem,procLabel) = (NoopProcessor<Path>(),CategoryProcessor())
//let sld = SplitLabeledData(sd, fromFunc: parentLabeler, procItem: &procItem, procLabel: &procLabel)
//var rawData = sld.toDataBunch(itemToTensor: pathsToTensor, labelToTensor: intsToTensor)
//let data = transformData(rawData, tfmItem: { openAndResize(fname: $0, size: 128) })

In [ ]:
let data = mnistDataBunch(flat: true)

In [ ]:
let (n,m) = (60000,784)
let c = 10
let nHid = 50

In [ ]:
func modelInit() -> BasicModel {return BasicModel(nIn: m, nHid: nHid, nOut: c)}

## Stateful optimizer

In [ ]:
//export
//Expandable enum to have easier names than LearningRate.self.
public struct HyperParams {
    public static let lr = LearningRate.self
}

In [ ]:
let hd = HeterogeneousDictionary()

In [ ]:
hd[HyperParams.lr]

0.4


In [ ]:
//export
open class StatDelegate {
    open var name: String { return "" }
    var defaultConfig: HeterogeneousDictionary { return HeterogeneousDictionary() }
    public init() {}
    public func update(
        state: inout [String: Tensor<Float>],
        for param: Tensor<Float>,
        along direction: Tensor<Float>,
        config: inout HeterogeneousDictionary
    ) { }
}

//export
open class StepDelegate {
    var defaultConfig: HeterogeneousDictionary { return HeterogeneousDictionary() }
    public init() {}
    public func update(
        param: inout Tensor<Float>,
        along direction: inout Tensor<Float>,
        state: [String: Tensor<Float>],
        config: inout HeterogeneousDictionary
    ) { }
}

In [ ]:
//export
public class StatefulOptimizer<Model: Layer>
    where Model.AllDifferentiableVariables == Model.CotangentVector{
    public var configs: [HeterogeneousDictionary]
    public var splitFunc: (Int) -> Int
    public var states: [String: Model.AllDifferentiableVariables]
    public var statDelegates: [StatDelegate]
    public var stepDelegates: [StepDelegate]
    public init(
        for model: __shared Model,
        stepDelegates: [StepDelegate],
        statDelegates: [StatDelegate],
        configs: [HeterogeneousDictionary],
        splitFunc: @escaping (Int) -> Int
    ) {
        self.configs = Array(repeating: HeterogeneousDictionary(), count: configs.count)
        states = [:]
        for stepDelegate in stepDelegates {
            for i in self.configs.indices { self.configs[i].merge(stepDelegate.defaultConfig) { (_, new) in new } }
        }
        for statDelegate in statDelegates {
            for i in self.configs.indices { self.configs[i].merge(statDelegate.defaultConfig) { (_, new) in new } }
            states[statDelegate.name] = Model.AllDifferentiableVariables.zero
        }
        for i in 0..<configs.count {
            self.configs[i].merge(configs[i]) { (_, new) in new }
        }
        self.stepDelegates = stepDelegates
        self.statDelegates = statDelegates
        self.splitFunc = splitFunc
    }
        
    public func update(
        _ model: inout Model.AllDifferentiableVariables,
        along direction: Model.CotangentVector
    ) {
        for (i,kp) in model.keyPaths.enumerated() {
            var grad = direction[keyPath: kp]
            var state = states.mapValues(){$0[keyPath: kp]}
            var config = configs[splitFunc(i)]
            for statDelegate in statDelegates {
                statDelegate.update(
                    state: &state,
                    for: model[keyPath: kp],
                    along: grad,
                    config: &config
                )
            }
            for n in states.keys { states[n]![keyPath: kp] = state[n]! }
            for stepDelegate in stepDelegates {
                stepDelegate.update(
                    param: &model[keyPath: kp],
                    along: &grad,
                    state: state,
                    config: &config
                )
            }
            configs[splitFunc(i)] = config
        }
    }
}

Conformance to the optimizer protocol

In [ ]:
//export
extension StatefulOptimizer: Optimizer{
    public var learningRate: Float {
        get { return configs.last![HyperParams.lr] } 
        set { 
            for i in configs.indices {self.configs[i][HyperParams.lr] = newValue }
        }
    }
    public var learningRates: [Float] {
        get {
            var res: [Float] = []
            for config in configs {res.append(config[HyperParams.lr])}
            return res
        }
        set { 
            for i in configs.indices {self.configs[i][HyperParams.lr] = newValue[i] } 
        }
    }
}

Convenience init when there are no param groups

In [ ]:
//export
extension StatefulOptimizer{
    public convenience init (for model: __shared Model,
                             stepDelegates: [StepDelegate],
                             statDelegates: [StatDelegate],
                             config: HeterogeneousDictionary) {
        self.init(for: model,
                  stepDelegates: stepDelegates,
                  statDelegates: statDelegates,
                  configs: [config],
                  splitFunc: { _ in return 0 })
    }
}

In [ ]:
//export
public class SGDStep: StepDelegate {
    override public func update(
        param: inout Tensor<Float>,
        along direction: inout Tensor<Float>,
        state: [String: Tensor<Float>],
        config: inout HeterogeneousDictionary
    ) {
        param -= direction * config[HyperParams.lr]
    }
}

Test discriminative learning rates/freezing

In [ ]:
func splitFunc(_ a: Int) -> Int { return a < 2 ? 0 : 1 }

In [ ]:
var configs = [HeterogeneousDictionary(HyperParams.lr, 0.0), HeterogeneousDictionary(HyperParams.lr, 0.01)]
func optFunc(_ model: BasicModel) -> StatefulOptimizer<BasicModel> {
    return StatefulOptimizer(
        for: model,
        stepDelegates: [SGDStep()],
        statDelegates: [],
        configs: configs,
        splitFunc: splitFunc)
}

In [ ]:
let learner = Learner(data: data, lossFunc: softmaxCrossEntropy, optFunc: optFunc, modelInit: modelInit)
let recorder = learner.makeDefaultDelegates(metrics: [accuracy])
learner.delegates.append(learner.makeNormalize(mean: mnistStats.mean, std: mnistStats.std))

In [ ]:
let params = learner.model.allDifferentiableVariables

In [ ]:
for kp in params.recursivelyAllWritableKeyPaths(to: TF.self) { 
    print(params[keyPath: kp][0]) 
}

[  0.035641838,    0.02483342,   -0.07116243,   -0.03510739,  -0.020960862,   -0.05498522,
    0.05191987,    0.01615886,    0.06687938,   -0.05391738,   0.062395394,    0.06987975,
  -0.016558869, -0.0008535976,   0.053457327,  -0.007723056,  -0.072978705,   -0.08286889,
  -0.050846554,    0.07266115,   -0.05838936,  -0.008778322,   0.027720077,   -0.05163173,
  -0.058223292,  -0.069665246,    0.07033115,  0.0011013018,   0.014139239,    0.07227897,
   0.058027793,   0.037321024,    0.07505389,   -0.04497029,   0.027333839,  -0.072806604,
  0.0035239554, -0.0020111178,   0.005757752,   0.012076079,   -0.07523716,   -0.06712338,
    0.07636316,   -0.07622585,  -0.017690504,   0.068187244,    0.08206651,  -0.046752308,
    0.03887641,   0.045985304]
0.0
[  0.18579277,  -0.18539035,  -0.28362188,   0.23897125,   0.21943688,  -0.30082512,
 -0.054752823,    0.2941639,  0.014344044,  -0.13743345]
0.0


In [ ]:
learner.fit(2)

In [ ]:
let params = learner.model.allDifferentiableVariables
for kp in params.recursivelyAllWritableKeyPaths(to: TF.self) { 
    print(params[keyPath: kp][0]) 
}

[  0.035641838,    0.02483342,   -0.07116243,   -0.03510739,  -0.020960862,   -0.05498522,
    0.05191987,    0.01615886,    0.06687938,   -0.05391738,   0.062395394,    0.06987975,
  -0.016558869, -0.0008535976,   0.053457327,  -0.007723056,  -0.072978705,   -0.08286889,
  -0.050846554,    0.07266115,   -0.05838936,  -0.008778322,   0.027720077,   -0.05163173,
  -0.058223292,  -0.069665246,    0.07033115,  0.0011013018,   0.014139239,    0.07227897,
   0.058027793,   0.037321024,    0.07505389,   -0.04497029,   0.027333839,  -0.072806604,
  0.0035239554, -0.0020111178,   0.005757752,   0.012076079,   -0.07523716,   -0.06712338,
    0.07636316,   -0.07622585,  -0.017690504,   0.068187244,    0.08206651,  -0.046752308,
    0.03887641,   0.045985304]
0.0
[  0.18579277,  -0.18539035,  -0.28362188,   0.23897125,   0.21943688,  -0.30082512,
 -0.054752823,    0.2941639,  0.014344044,  -0.13743345]
0.0


In [ ]:
//export
public struct WeightDecayKey: HetDictKey { public static var defaultValue: Float = 0 }

public extension HyperParams {
    static let wd = WeightDecayKey.self
}

public class WeightDecay: StepDelegate {
    override public func update(
        param: inout Tensor<Float>,
        along direction: inout Tensor<Float>,
        state: [String: Tensor<Float>],
        config: inout HeterogeneousDictionary
    ) {
        param *= 1 - config[HyperParams.lr] * config[HyperParams.wd]
    }
}

In [ ]:
//export
public class L2Regularization: StepDelegate {
    override public func update(
        param: inout Tensor<Float>,
        along direction: inout Tensor<Float>,
        state: [String: Tensor<Float>],
        config: inout HeterogeneousDictionary
    ) {
        direction += config[HyperParams.wd] * param
    }
}

In [ ]:
//export
//Expandable enum to have tab completes/typo-proof for state variable names.
public struct StateKeys {
    public static let avgGrad = "averageGrad"
}

In [ ]:
//export

public struct Momentum: HetDictKey { public static var defaultValue: Float = 0.9 }
public struct MomentumDampening: HetDictKey, Equatable { public static var defaultValue: Float = 0.1 }
public extension HyperParams {
    static let mom = Momentum.self
    static let momDamp = MomentumDampening.self
}

public class AverageGrad: StatDelegate {
    public let dampened: Bool
    public init(dampened: Bool = false) { self.dampened = dampened }
    override public var name: String { return StateKeys.avgGrad }
    override public func update(
        state: inout [String: Tensor<Float>],
        for param: Tensor<Float>,
        along direction: Tensor<Float>,
        config: inout HeterogeneousDictionary
    ) {
        state[StateKeys.avgGrad]! *= config[HyperParams.mom]
        config[HyperParams.momDamp] = 1.0 - (dampened ? config[HyperParams.mom] : 0.0)
        state[StateKeys.avgGrad]! += config[HyperParams.momDamp] * direction
    }
}

In [ ]:
//export
public class MomentumStep: StepDelegate {
    override public func update(
        param: inout Tensor<Float>,
        along direction: inout Tensor<Float>,
        state: [String: Tensor<Float>],
        config: inout HeterogeneousDictionary
    ) {
        param -= state[StateKeys.avgGrad]! * config[HyperParams.lr]
    }
}

In [ ]:
func optFunc(_ model: BasicModel) -> StatefulOptimizer<BasicModel> {
    return StatefulOptimizer(
        for: model,
        stepDelegates: [MomentumStep()],
        statDelegates: [AverageGrad()],
        config: HeterogeneousDictionary(HyperParams.lr, 0.01))
}

In [ ]:
let learner = Learner(data: data, lossFunc: softmaxCrossEntropy, optFunc: optFunc, modelInit: modelInit)
let recorder = learner.makeDefaultDelegates(metrics: [accuracy])
learner.delegates.append(learner.makeNormalize(mean: mnistStats.mean, std: mnistStats.std))

In [ ]:
learner.fit(2)

In [ ]:
//export
public struct SquareMomentum: HetDictKey { public static var defaultValue: Float = 0.99 }
public struct SquareMomentumDampening: HetDictKey { public static var defaultValue: Float = 0.99 }

public extension HyperParams {
    static let ²mom = Momentum.self
    static let ²momDamp = MomentumDampening.self
}

public extension StateKeys {
    static let avgSqr = "averageSquaredGrad"
}

public class AverageSquaredGrad: StatDelegate {
    let dampened: Bool
    public init(dampened: Bool = true) { self.dampened = dampened }
    override public var name: String { return StateKeys.avgSqr }
    override public func update(
        state: inout [String: Tensor<Float>],
        for param: Tensor<Float>,
        along direction: Tensor<Float>,
        config: inout HeterogeneousDictionary
    ) {
        state[StateKeys.avgSqr]! *= config[HyperParams.²mom]
        config[HyperParams.²momDamp] = 1.0 - (dampened ? config[HyperParams.²mom] : 0.0)
        state[StateKeys.avgSqr]! += config[HyperParams.²momDamp] * direction.squared()
    }
}

In [ ]:
//export
public extension StateKeys {
    static let step = "stepCount"
}

public class StepCount: StatDelegate {
    override public var name: String { return StateKeys.step }
    override public func update(
        state: inout [String: Tensor<Float>],
        for param: Tensor<Float>,
        along direction: Tensor<Float>,
        config: inout HeterogeneousDictionary
    ) {
        state[StateKeys.step]! += 1.0
    }
}

In [ ]:
//export
public struct Epsilon: HetDictKey { public static var defaultValue: Float = 1e-5 }
public extension HyperParams {
    static let eps = Epsilon.self
}

In [ ]:
//export
public class AdamStep: StepDelegate {
    override public func update(
        param: inout Tensor<Float>,
        along direction: inout Tensor<Float>,
        state: [String: Tensor<Float>],
        config: inout HeterogeneousDictionary
    ) {
        let step = state[StateKeys.step]!
        let (mom,damp) = (config[HyperParams.mom],config[HyperParams.momDamp])
        let debias1 = damp * (1 - pow(mom, step)) / (1 - mom)
        let num = debias1 * state[StateKeys.avgGrad]!
        
        let (²mom,²damp) = (config[HyperParams.²mom],config[HyperParams.²momDamp])
        let debias2 = ²damp * (1 - pow(²mom, step)) / (1 - ²mom)
        let denom = sqrt(state[StateKeys.avgSqr]!/debias2) + config[HyperParams.eps]
        
        param -= config[HyperParams.lr] * num / denom
    }
}

In [ ]:
func optFunc(_ model: BasicModel) -> StatefulOptimizer<BasicModel> {
    return StatefulOptimizer(
        for: model,
        stepDelegates: [AdamStep()], 
        statDelegates: [AverageGrad(dampened: true), AverageSquaredGrad(), StepCount()], 
        config: HeterogeneousDictionary(HyperParams.lr, 0.01))
}

In [ ]:
let learner = Learner(data: data, lossFunc: softmaxCrossEntropy, optFunc: optFunc, modelInit: modelInit)
let recorder = learner.makeDefaultDelegates(metrics: [accuracy])
learner.delegates.append(learner.makeNormalize(mean: mnistStats.mean, std: mnistStats.std))

In [ ]:
learner.fit(2)

In [ ]:
class LambStep: StepDelegate {
    override var defaultConfig: HeterogeneousDictionary {
        return HeterogeneousDictionary(HyperParams.eps, 1e-6, HyperParams.wd, 0.0)
    }
    override func update(
        param: inout Tensor<Float>,
        along direction: inout Tensor<Float>,
        state: [String: Tensor<Float>],
        config: inout HeterogeneousDictionary
    ) {
        let stepCount = state[StateKeys.step]!
        let (mom,damp) = (config[HyperParams.mom],config[HyperParams.momDamp])
        let debias1 = damp * (1 - pow(mom, stepCount)) / (1 - mom)
        let num = debias1 * state[StateKeys.avgGrad]!
        
        let (²mom,²damp) = (config[HyperParams.²mom],config[HyperParams.²momDamp])
        let debias2 = ²damp * (1 - pow(²mom, stepCount)) / (1 - ²mom)
        let denom = sqrt(state[StateKeys.avgSqr]!/debias2) + config[HyperParams.eps]
        
        let step = num / denom + config[HyperParams.wd] * param
        let r1 = sqrt((param * param).mean())
        let r2 = sqrt((step * step).mean())
        let factor = min(r1 / r2, Float(10.0))
        param -= config[HyperParams.lr] * factor * step
    }
}

### Making convenienc functions

In [ ]:
// export
public func SGDOpt<Model>(lr: Float, mom: Float = 0.9, wd: Float = 0.0, dampening: Bool = false
                         ) -> ((Model) -> StatefulOptimizer<Model>) {
    var steppers = (mom != 0) ? [MomentumStep()] : [SGDStep()]
    if wd != 0 { steppers.append(WeightDecay()) }
    let stats = (mom != 0) ? [AverageGrad(dampened: dampening)] : []
    var config = HeterogeneousDictionary(HyperParams.lr, lr)
    if mom != 0 { config[HyperParams.mom] = mom }
    if wd != 0  { config[HyperParams.wd ] = wd  }
    return {model in 
        return StatefulOptimizer(for: model, stepDelegates: steppers, statDelegates: stats, config: config)}
}

In [ ]:
// export
public func AdamOpt<Model>(for model: Model, lr: Float, mom: Float = 0.9, beta: Float=0.99, wd: Float = 0.0, eps: Float = 1e-5
                         ) -> ((Model) -> StatefulOptimizer<Model>) {
    var steppers: [StepDelegate] = [AdamStep()]
    if wd != 0 { steppers.append(WeightDecay()) }
    let stats = [AverageGrad(dampened: true), AverageSquaredGrad(), StepCount()]
    var config = HeterogeneousDictionary(HyperParams.lr, lr)
    config[HyperParams.mom] = mom
    config[HyperParams.²mom] = beta
    config[HyperParams.eps] = eps
    if wd != 0  { config[HyperParams.wd ] = wd  }
    return {model in 
        return StatefulOptimizer(for: model, stepDelegates: steppers, statDelegates: stats, config: config)}
}

## Export

In [ ]:
notebookToScript(fname: Path.cwd / "09_optimizer.ipynb")